## Выбор фреймворка




Используйте ноутбук, чтобы по шагам запускать загрузку моков, выбор фреймворка и кодогенерацию.

In [12]:
%load_ext autoreload
%autoreload 2


In [3]:
from mas_automl.code_agent.load_mocks import load_mock_inputs

DATA_ANALYZE, METADATA, REGISTRY, FINAL_DATA = load_mock_inputs()
print(f"Ключи data: {list[str](DATA_ANALYZE.keys())}")
print(f"Ключи metadata: {list(METADATA.keys())}")
print(f"Фреймворки: {list(REGISTRY.keys())}")


Ключи data: ['summary', 'priority', 'steps', 'example_pipeline_snippet', 'frameworks_recommended', 'rationale', 'estimated_complexity', 'confidence']
Ключи metadata: ['dataset_id', 'name', 'version', 'version_label', 'description', 'citation', 'paper_url', 'paper_reference', 'creator', 'contributor', 'collection_date', 'upload_date', 'language', 'licence', 'url', 'original_data_url', 'minio_url', 'format', 'file_id', 'default_target_attribute', 'ignore_attribute', 'row_id_attribute', 'num_rows', 'num_features', 'num_classes', 'num_missing_values', 'quality', 'dataset_type', 'tags', 'status', 'visibility', 'extra_info', 'local_path']
Фреймворки: ['AutoGluon', 'H2O AutoML', 'LightAutoML']


In [4]:
from mas_automl.code_agent.load_mocks import load_mock_inputs
from IPython.display import display, Markdown

DATA_ANALYZE, METADATA, REGISTRY, FINAL_DATA = load_mock_inputs()

# Pretty display in notebook
display(Markdown(f"""
### Loaded Mock Inputs

#### Data Analyze Keys
{', '.join(DATA_ANALYZE.keys())}

#### Metadata Keys
{', '.join(METADATA.keys())}

#### Registry Frameworks
{', '.join(REGISTRY.keys())}

{', '.join(FINAL_DATA.keys())}
"""))





### Loaded Mock Inputs

#### Data Analyze Keys
summary, priority, steps, example_pipeline_snippet, frameworks_recommended, rationale, estimated_complexity, confidence

#### Metadata Keys
dataset_id, name, version, version_label, description, citation, paper_url, paper_reference, creator, contributor, collection_date, upload_date, language, licence, url, original_data_url, minio_url, format, file_id, default_target_attribute, ignore_attribute, row_id_attribute, num_rows, num_features, num_classes, num_missing_values, quality, dataset_type, tags, status, visibility, extra_info, local_path

#### Registry Frameworks
AutoGluon, H2O AutoML, LightAutoML

manifest, validation_report, metafeatures, preprocessing_recipe, code_agent_recommendation, run_metadata


In [5]:
# DATA_ANALYZE, METADATA, REGISTRY
import pandas as pd

PATH_TO_CSV = FINAL_DATA["manifest"]["local_path"]
dataset_df = pd.read_csv(PATH_TO_CSV)

dataset_df.info(), dataset_df.head(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 21 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   checking_status         1000 non-null   object 
 1   duration                1000 non-null   int64  
 2   credit_history          1000 non-null   object 
 3   purpose                 1000 non-null   object 
 4   credit_amount           1000 non-null   float64
 5   savings_status          1000 non-null   object 
 6   employment              1000 non-null   object 
 7   installment_commitment  1000 non-null   int64  
 8   personal_status         1000 non-null   object 
 9   other_parties           1000 non-null   object 
 10  residence_since         1000 non-null   int64  
 11  property_magnitude      1000 non-null   object 
 12  age                     1000 non-null   int64  
 13  other_payment_plans     1000 non-null   object 
 14  housing                 1000 non-null   o

(None,
   checking_status  duration                  credit_history   purpose  \
 0              <0         6  critical/other existing credit  radio/tv   
 1        0<=X<200        48                   existing paid  radio/tv   
 
    credit_amount    savings_status employment  installment_commitment  \
 0         1169.0  no known savings        >=7                       4   
 1         5951.0              <100     1<=X<4                       2   
 
       personal_status other_parties  ...  property_magnitude age  \
 0         male single          none  ...         real estate  67   
 1  female div/dep/mar          none  ...         real estate  22   
 
    other_payment_plans housing existing_credits      job num_dependents  \
 0                 none     own                2  skilled              1   
 1                 none     own                1  skilled              1   
 
    own_telephone foreign_worker class  
 0            yes            yes  good  
 1           none       

In [6]:
# --- Инициализация клиента ---
from mas_automl.code_agent.openai_wraper import LLMClient, LLMConfig

client = LLMClient(LLMConfig())

# --- Вызов выбора фреймворка ---


{'api_key': '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'}


In [7]:
client.chat('Health-check. Are you alive? short answer')

"Yes, I'm here! How can I assist you today?"

## Выбор фреймворка

## Итеративная генерация и тестирование кода

Ниже показан процесс итеративной генерации кода с тестированием через execnet_gateway


In [8]:
# Итеративная генерация и тестирование кода
from mas_automl.code_agent.base_pipeline import generate_code, evaluate_code
from mas_automl.code_agent.execnet_gateway import PythonSandboxClient
from pathlib import Path
import pandas as pd

CSV_PATH = FINAL_DATA["manifest"]["local_path"]
OUTPUT_DIR = str(Path(CSV_PATH).parent / "predictions")
Path(OUTPUT_DIR).mkdir(parents=True, exist_ok=True)

framework = "scikit-learn"
max_iterations = 3
feedback = ""

print(f"CSV путь: {CSV_PATH}")
print(f"Директория для предиктов: {OUTPUT_DIR}\n")



CSV путь: C:\Users\User1\Desktop\ITMO_bootcamp\data\datasets\openml_31_credit-g.csv
Директория для предиктов: C:\Users\User1\Desktop\ITMO_bootcamp\data\datasets\predictions



In [7]:

for iteration in range(1, max_iterations + 1):
    print(f"{'='*60}")
    print(f"Итерация {iteration}/{max_iterations}")
    print(f"{'='*60}\n")
    
    # Генерируем код
    print("Генерация кода...")
    code = generate_code(framework, client, iteration, feedback, FINAL_DATA)
    print(f"Сгенерированный код:\n{code}\n")
    
    # Тестируем код
    print("Тестирование кода...")
    tests_passed, test_feedback, predict_path = evaluate_code(
        code, framework, CSV_PATH, OUTPUT_DIR, iteration, FINAL_DATA
    )
    
    print(f"Тесты пройдены: {tests_passed}")
    print(f"Обратная связь: {test_feedback}")
    if predict_path:
        print(f"Путь к предиктам: {predict_path}")
        # Загружаем и показываем предикты
        try:
            pred_df = pd.read_csv(predict_path)
            print(f"\nПервые 5 строк предиктов:")
            print(pred_df.head())
        except Exception as e:
            print(f"Ошибка загрузки предиктов: {e}")
    
    print("\n")
    
    if tests_passed:
        print("✅ Все тесты пройдены! Останавливаем итерации.")
        break
    else:
        feedback = test_feedback
        print("❌ Тесты не пройдены. Переходим к следующей итерации с учетом обратной связи.\n")

print(f"\n{'='*60}")
print("Итоговый результат:")
print(f"Итераций выполнено: {iteration}")
print(f"Тесты пройдены: {tests_passed}")
print(f"Путь к предиктам: {predict_path}")

Итерация 1/3

Генерация кода...


2025-11-14 15:23:35,785 [INFO] Creating new execnet gateway with python=c:\Users\User1\Desktop\ITMO_bootcamp\.venv\Scripts\python.exe
2025-11-14 15:23:35,859 [INFO] Gateway and channel created.


Сгенерированный код:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split

def train_model(train_df: pd.DataFrame, test_df: pd.DataFrame, label: str):
    numeric_features = ['duration', 'credit_amount', 'installment_commitment', 'residence_since', 'age', 'existing_credits', 'num_dependents']
    categorical_features = ['checking_status', 'credit_history', 'purpose', 'savings_status', 'employment', 'personal_status', 'other_parties', 'property_magnitude', 'other_payment_plans', 'housing', 'job', 'own_telephone', 'foreign_worker']
    
    X_train = train_df.drop(columns=[label])
    y_train = train_df[label]
    
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', StandardScaler(), numeric_features),
            ('cat', OneHotEncoder(), cate

NameError: name 'result' is not defined

## Просмотр сохраненных предиктов

Если предикты были успешно сохранены, можно загрузить и проанализировать их


In [ ]:
# Просмотр сохраненных предиктов
from pathlib import Path
import pandas as pd

if predict_path and Path(predict_path).exists():
    pred_df = pd.read_csv(predict_path)
    print(f"Загружено предиктов: {len(pred_df)}")
    print(f"\nПервые 10 строк:")
    print(pred_df.head(10))
    print(f"\nСтатистика:")
    print(pred_df.describe())
    
    # Простая метрика точности
    if 'true_label' in pred_df.columns and 'prediction' in pred_df.columns:
        from sklearn.metrics import accuracy_score
        accuracy = accuracy_score(pred_df['true_label'], pred_df['prediction'])
        print(f"\nТочность (accuracy): {accuracy:.4f}")
else:
    print("Предикты не найдены. Запустите предыдущую ячейку для генерации.")


# Кодогенерация

In [7]:
from typing import Dict, Any
def generate_code(
    framework: str, llm: LLMClient, iteration: int, feedback: str, final_data: Dict[str, Any] | None = None
) -> str:
    """Генерирует код для обучения модели."""
    preprocessing_info = ""
    if final_data:
        preprocessing_recipe = final_data.get("preprocessing_recipe", {})
        if preprocessing_recipe:
            preprocessing_info = (
                f"\n\nИнформация о препроцессинге:\n"
                f"- Числовые колонки: {preprocessing_recipe.get('numeric_columns', [])}\n"
                f"- Категориальные колонки: {preprocessing_recipe.get('categorical_columns', [])}\n"
                f"- Тип задачи: {preprocessing_recipe.get('task_type', 'classification')}\n"
            )
    
    prompt = f"""
    Итерация {iteration}.
    Ты — опытный Data Scientist. Нужно написать корректный код,
    который использует МОЙ AutoML модуль (он уже импортирован; НЕ нужно определять класс AutoML).
    Действуй так, будто class_labels и label определен и инициализирован!
    
    Код должен:
    1) Использовать:
        - df : pandas.DataFrame

    2) Правильно разделять данные:
        X =  df.drop(columns=[label])
        y =  df[label]
        где - label : str (метка класса)

    3) Написать этот код для labels:
        label_mapping = {{v: k for k, v in enumerate(class_labels)}}
        if isinstance(y, pd.Series):
            y = np.array(y.map(label_mapping), dtype=int)
        else:
            # Если y уже массив или другой тип, преобразуем через pandas для совместимости
            y_series = pd.Series(y)
            y = np.array(y_series.map(label_mapping), dtype=int)


    4) Инициализировать AutoML с параметрами, например:
        automl = AutoML(
            task='classification',
            use_preprocessing_pipeline=False,
            feature_selector_type=None,
            use_val_test_pipeline=False,
            auto_models_init_kwargs={{ 
                "metric": "roc_auc",
                "time_series": False,
                "models_list": ["linear", "forests", "boostings"],
                "blend": True,
                "stack": True,
                "n_splits": 10
            }},
            n_jobs=3,
            random_state=0,
        )

    Описание параметров класса AutoML:
     Parameters
    ----------
    task
        Machine Learning task to solve, by default classification
        Should be one of:
            -"classification (binary and multiclass)
            - regression
    use_preprocessing_pipeline, optional
        Whether to use preprocessing pipeline, by default True
    preprocessing_pipeline_kwargs, optional
        Keyword arguments to initialize preprocessing_pipeline, by default {{ }}
        List of possible arguments and their default values:
            - pipe_steps = ['all']
            - nan_share_ts=0.2
            - qconst_feature_val_share_ts=0.95
            - impute_num_strategy='median'
            - impute_cat_strategy='most_frequent'
            - outlier_capping_method='gaussian'
            - outlier_cap_tail='both'
            - corr_ts = 0.8
            - corr_coef_methods=['pearson', 'spearman']
            - corr_selection_method="missing_values"
            - oe_min_freq=0.1
            - obj_encoders = ['oe', 'ohe']
            - num_encoder = "ss"
            - verbose=True
    use_val_test_pipeline, optional
        Whether to use val_test_pipeline, by default True
    val_test_pipeline_kwargs, optional
        Keyword arguments to initialize val_test_pipeline, by default {{ }}
        List of possible arguments and their default values:
            - pipe_steps = ['all']
            - split_col='is_test_for_val'
            - psi_cut_off=0.5
            - psi_threshold=0.2
            - psi_bins=15
            - psi_strategy='equal_width'
            - adversarial_auc_trshld=0.7
            - verbose=True
    feature_selector_type, optional
        Whether to use feature_selector and which feature_selector to use, by default "CatboostByShap"
        Possible values:
            - CatboostByShap
            - LAMA
    feature_selector_kwargs, optional
        Keyword arguments to initialize feature_selector, by default {{ }}
        List of possible arguments and their default values:
            * CatboostByShap
                - n_features_to_select = 50
                - complexity = "Regular"
                - steps = 5
            * LAMA
                -task_type
                - target_colname
                - metric_name
                - metric_direction
                - timeout=120
                - model='lama'
                - strategy='RFA'
                - permutation_n_repeats = 5
    auto_models_init_kwargs, optional
        Keyword arguments to initialize AutoModel, by default {{ }}
        List of possible arguments and their default values:
            - metric
            - time_series=False
            - models_list=None
            - blend=False
            - stack=False
            - timeout=60 (timeout для моделей, которые его поддерживают, например TabularLama)
    n_jobs, optional
        Number of cores for parallel computations, by default 1
    random_state, optional
        Random state, by default 42
    log_to_file, optional
        Whether to save logs in files.
        Save files locations:
            - ml_data/YYYY_mm_dd___HH-MM-SS/logs.log for info logs
            - ml_data/YYYY_mm_dd___HH-MM-SS/error.log for error, critical, warning logs

    5) Обучить модель:
        automl = automl.fit(
            X, y,
            auto_model_fit_kwargs={{"tuning_timeout": 10}}
        )

        — НЕ использовать sklearn в любом виде
        — НЕ выполнять вручную препроцессинг, кодирование, скейлинг

    6) Получить предсказания:
        preds = automl.predict(X)

    7) Вычислить переменные:
        score = automl.auto_model.best_score
        test_predictions = preds[:, 1]


    Если передано preprocessing_recipe — учитывай его только как информационный блок,
    но НЕ применяй его в коде.

    Информация о препроцессинге:
    {preprocessing_info}

    ПРИМЕР КАК ИСПОЛЬЗУЕТСЯ МОЙ AutoML (НЕ копировать, только ориентир):
        label_mapping = {{v: k for k, v in enumerate(class_labels)}}
        if isinstance(y, pd.Series):
            y = np.array(y.map(label_mapping), dtype=int)
        else:
            # Если y уже массив или другой тип, преобразуем через pandas для совместимости
            y_series = pd.Series(y)
            y = np.array(y_series.map(label_mapping), dtype=int)

        automl = AutoML(...)
        automl = automl.fit(X, y, auto_model_fit_kwargs={{"tuning_timeout": 10}})
        score = automl.auto_model.best_score
        test_predictions = automl.predict(X)[:, 1]



    Требования к выводу:
    - Вернуть СТРОГО: только код функции + необходимые импорты.
    - Никакого текста, комментариев, markdown.
    - Переменные: score и test_predictions — обязательны.
    - Код я буду исполнять - НЕ ПИШИ ФУНКЦИЮ.
    - Действуй так, будто class_labels и label определен и инициализирован!
    

    Обратная связь:
    {feedback or "нет"}
    """


    fallback_code = _fallback_code_template_sklearn(final_data)
    raw_code = llm.chat(prompt, fallback=fallback_code)
    return _extract_code(raw_code) or fallback_code

In [8]:
def _fallback_code_template_sklearn(final_data: Dict[str, Any] | None = None) -> str:
    """Шаблон кода на scikit-learn для fallback."""
    numeric_cols = ["duration", "credit_amount", "installment_commitment", "residence_since", "age", "existing_credits", "num_dependents"]
    categorical_cols = ["checking_status", "credit_history", "purpose", "savings_status", "employment", "personal_status", "other_parties", "property_magnitude", "other_payment_plans", "housing", "job", "own_telephone", "foreign_worker"]
    
    if final_data:
        preprocessing_recipe = final_data.get("preprocessing_recipe", {})
        if preprocessing_recipe:
            numeric_cols = preprocessing_recipe.get("numeric_columns", numeric_cols)
            categorical_cols = preprocessing_recipe.get("categorical_columns", categorical_cols)
    
    return (
        "import pandas as pd\n"
        "import numpy as np\n"
        "from sklearn.compose import ColumnTransformer\n"
        "from sklearn.pipeline import Pipeline\n"
        "from sklearn.preprocessing import StandardScaler, OneHotEncoder\n"
        "from sklearn.impute import SimpleImputer\n"
        "from sklearn.ensemble import RandomForestClassifier\n\n"
        f"numeric_cols = {numeric_cols}\n"
        f"categorical_cols = {categorical_cols}\n\n"
        "def train_model(train_df: pd.DataFrame, test_df: pd.DataFrame, label: str):\n"
        "    # Подготовка признаков\n"
        "    numeric_transformer = Pipeline(steps=[\n"
        "        ('imputer', SimpleImputer(strategy='median')),\n"
        "        ('scaler', StandardScaler())\n"
        "    ])\n"
        "    categorical_transformer = Pipeline(steps=[\n"
        "        ('imputer', SimpleImputer(strategy='most_frequent')),\n"
        "        ('encoder', OneHotEncoder(handle_unknown='ignore', sparse_output=False))\n"
        "    ])\n"
        "    preprocessor = ColumnTransformer(\n"
        "        transformers=[\n"
        "            ('num', numeric_transformer, numeric_cols),\n"
        "            ('cat', categorical_transformer, categorical_cols)\n"
        "        ]\n"
        "    )\n"
        "    \n"
        "    # Подготовка данных\n"
        "    X_train = train_df.drop(columns=[label])\n"
        "    y_train = train_df[label]\n"
        "    \n"
        "    # Создание пайплайна\n"
        "    model = Pipeline(steps=[\n"
        "        ('preprocessor', preprocessor),\n"
        "        ('classifier', RandomForestClassifier(n_estimators=100, random_state=42, max_depth=10))\n"
        "    ])\n"
        "    \n"
        "    # Обучение\n"
        "    model.fit(X_train, y_train)\n"
        "    \n"
        "    return model\n"
    )


def _extract_code(raw_output: str) -> str:
    if "```" not in raw_output:
        return raw_output.strip()
    parts = raw_output.split("```")
    if len(parts) < 3:
        return raw_output.strip()
    code_block = parts[1]
    if code_block.startswith(("python", "py")):
        code_block = code_block.split("\n", 1)[1]
    return code_block.strip()


In [9]:
iteration = 1
code = generate_code(framework, client, iteration, feedback, FINAL_DATA)


In [45]:
code

'import pandas as pd\nimport numpy as np\n\nlabel_mapping = {v: k for k, v in enumerate(class_labels)}\nif isinstance(y, pd.Series):\n    y = np.array(y.map(label_mapping), dtype=int)\nelse:\n    y_series = pd.Series(y)\n    y = np.array(y_series.map(label_mapping), dtype=int)\n\nX = df.drop(columns=[label])\ny = df[label]\n\nautoml = AutoML(\n    task=\'classification\',\n    use_preprocessing_pipeline=False,\n    feature_selector_type=None,\n    use_val_test_pipeline=False,\n    auto_models_init_kwargs={ \n        "metric": "roc_auc",\n        "time_series": False,\n        "models_list": ["linear", "forests", "boostings"],\n        "blend": True,\n        "stack": True,\n        "n_splits": 10\n    },\n    n_jobs=3,\n    random_state=0,\n)\n\nautoml = automl.fit(\n    X, y,\n    auto_model_fit_kwargs={"tuning_timeout": 10}\n)\n\nscore = automl.auto_model.best_score\npreds = automl.predict(df)\ntest_predictions = preds[:, 1]'

In [1]:
from __future__ import annotations

import json
import sys
from dataclasses import dataclass
from datetime import datetime
from pathlib import Path
from typing import Any, Dict, Tuple

if __package__ is None or __package__ == "":
    from mas_automl.code_agent.load_mocks import load_mock_inputs  # type: ignore
    from mas_automl.code_agent.openai_wraper import LLMClient, LLMConfig  # type: ignore
    from mas_automl.code_agent.execnet_gateway import PythonSandboxClient, SandboxResult  # type: ignore
else:
    from .load_mocks import load_mock_inputs
    from .openai_wraper import LLMClient, LLMConfig
    from .execnet_gateway import PythonSandboxClient, SandboxResult

DEFAULT_MAX_ITERATIONS = 3
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
CSV_PATH_TO_PREDICT = f"C:\\Users\\User1\\Desktop\\ITMO_bootcamp\\data\\datasets\\TEST\\test_{timestamp}.csv"

@dataclass
class PipelineResult:
    framework: str
    reason: str
    code: str
    tests_passed: bool
    iterations: int
    feedback: str
    predict_path: str | None = None


def evaluate_code(
    code: str,
    framework: str,
    csv_path: str,
    CSV_PATH_TO_PREDICT: str,
    output_dir: str,
    iteration: int,
    final_data: Dict[str, Any] | None,
    class_labels,
) :
    """
    Выполняет код в песочнице, тестирует функцию train_model и сохраняет предикты.
    Возвращает (tests_passed, feedback, predict_path).
    """
    sandbox = PythonSandboxClient.get()
    
    # Получаем информацию о датасете
    target_column = "class"
    if final_data:
        target_column = final_data.get("manifest", {}).get("target_column", "class")
    print("1 - start testing")
    # Подготавливаем код для выполнения
    test_code = f"""
import pandas as pd
import numpy as np
from pathlib import Path
import json
from datetime import datetime
from automl import AutoML

class_labels = {class_labels}
# Загружаем данные
df = pd.read_csv(r'{csv_path}')
print(f"Загружено строк: {{len(df)}}")
label = 'class'
# Генерированный код пользователя
{code}

# Выполняем функцию
try:
    df["test_predittions"] = test_predictions
    df.to_csv(r'{CSV_PATH_TO_PREDICT}', index=False)
    # print(f"Предикты сохранены в: {CSV_PATH_TO_PREDICT}")
    CSV_PATH_TO_PREDICT = r'{CSV_PATH_TO_PREDICT}'
    # Простые проверки
    errors = []
   
    if errors and any(errors):
        raise ValueError("; ".join([e for e in errors if e]))
    
    result = {{"ok": True, "predict_path": CSV_PATH_TO_PREDICT, "score": score, "message": "Все проверки пройдены"}}
    
except Exception as e:
    import traceback
    result = {{"ok": False, "predict_path": None, "score": score, "message": str(e), "traceback": traceback.format_exc()}}

# Выводим результат в формате JSON для парсинга
print("RESULT_START")
print(json.dumps(result, ensure_ascii=False))
print("RESULT_END")
"""
    
    # Выполняем код в песочнице
    result = sandbox.run(test_code)
    print("2 - end testing")
  
    if not result.ok:
        feedback = f"Ошибка выполнения: {result.stderr}\n{result.stdout}"
        return False, feedback, None, None
    print("3 - try gateway")

    # Парсим результат из stdout
    try:
        stdout = result.stdout
        # Ищем маркеры RESULT_START и RESULT_END
        if "RESULT_START" in stdout and "RESULT_END" in stdout:
            start_idx = stdout.find("RESULT_START") + len("RESULT_START")
            end_idx = stdout.find("RESULT_END")
            result_json = stdout[start_idx:end_idx].strip()
            result_dict = json.loads(result_json)
        else:
            # Fallback: пытаемся найти JSON в stdout
            import re
            json_match = re.search(r'\{[^{}]*"ok"[^{}]*\}', stdout)
            if json_match:
                result_dict = json.loads(json_match.group())
            else:
                result_dict = {"ok": False, "message": "Не удалось найти результат в stdout"}
    except Exception as e:
        feedback = f"Ошибка парсинга результата: {e}\nStdout: {result.stdout}\nStderr: {result.stderr}"
        return False, feedback, None, None
    print("4 - end gateway")

    if result_dict.get("ok", False):
        predict_path = result_dict.get("predict_path")
        score = result_dict.get("score")
        message = result_dict.get("message", "Проверки пройдены")
        return True, message, predict_path, score
    else:
        error_msg = result_dict.get("message", "Неизвестная ошибка")
        traceback_info = result_dict.get("traceback", "")
        score = result_dict.get("score")

        feedback = f"Тесты не пройдены: {error_msg}\n{traceback_info}"
        return False, feedback, None, None



In [2]:
# print("Генерация кода...")
# code = generate_code(framework, client, iteration, feedback, FINAL_DATA)
# print(f"Сгенерированный код:\n{code}\n")
from typing import Any


iteration = 1 
# Тестируем код
print("Тестирование кода...")
tests_passed, test_feedback, predict_path, score = evaluate_code(
    code, framework, CSV_PATH, CSV_PATH_TO_PREDICT, OUTPUT_DIR, iteration, FINAL_DATA,
    class_labels=list[Any](FINAL_DATA['metafeatures']['dataset']['class_balance'].keys())
)

predict_path = CSV_PATH_TO_PREDICT

print(f"Тесты пройдены: {tests_passed}")
print(f"Обратная связь: {test_feedback}")
if predict_path:
    print(f"Путь к предиктам: {predict_path}")
    # Загружаем и показываем предикты
    try:
        pred_df = pd.read_csv(predict_path)
        print(f"\nПервые 5 строк предиктов:")
        print(pred_df.head())
    except Exception as e:
        print(f"Ошибка загрузки предиктов: {e}")

print("\n")

Тестирование кода...


NameError: name 'code' is not defined

In [12]:
tests_passed, test_feedback, predict_path 

(False,
 'Тесты не пройдены: [Errno 22] Invalid argument: \'C:\\\\Users\\\\User1\\\\Desktop\\\\ITMO_bootcamp\\\\data\\\\datasets\\\\TEST\\\\test_2025-11-15 12:31:26.017693.csv\'\nTraceback (most recent call last):\n  File "<string>", line 57, in <module>\n  File "c:\\Users\\User1\\Desktop\\ITMO_bootcamp\\.venv\\Lib\\site-packages\\pandas\\util\\_decorators.py", line 333, in wrapper\n    return func(*args, **kwargs)\n           ^^^^^^^^^^^^^^^^^^^^^\n  File "c:\\Users\\User1\\Desktop\\ITMO_bootcamp\\.venv\\Lib\\site-packages\\pandas\\core\\generic.py", line 3989, in to_csv\n    return DataFrameRenderer(formatter).to_csv(\n           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^\n  File "c:\\Users\\User1\\Desktop\\ITMO_bootcamp\\.venv\\Lib\\site-packages\\pandas\\io\\formats\\format.py", line 1014, in to_csv\n    csv_formatter.save()\n  File "c:\\Users\\User1\\Desktop\\ITMO_bootcamp\\.venv\\Lib\\site-packages\\pandas\\io\\formats\\csvs.py", line 251, in save\n    with get_handle(\n         ^^^^^^

In [15]:
tests_passed, test_feedback, predict_path 

(False,
 'Тесты не пройдены: \'numpy.ndarray\' object has no attribute \'to_csv\'\nTraceback (most recent call last):\n  File "<string>", line 57, in <module>\nAttributeError: \'numpy.ndarray\' object has no attribute \'to_csv\'\n',
 'C:\\Users\\User1\\Desktop\\ITMO_bootcamp\\data\\datasets\\TEST\\test1__.csv')

In [ ]:
from IPython.display import display, Markdown

# Your existing call (unchanged)
from mas_automl.code_agent.base_pipeline import choose_framework

framework, reason, prompt = choose_framework(
    DATA_ANALYZE, METADATA, REGISTRY, FINAL_DATA, client)

# Pretty display in notebook
display(Markdown(f"""
### Selected Framework
**Framework:** {framework}

### Reasoning
{reason}

### Generated Prompt
{prompt}
"""))



### Selected Framework
**Framework:** H2O AutoML

### Reasoning
H2O AutoML является наиболее стабильным и универсальным выбором для табличных данных данного размера (1000 строк, 21 признаков) и типа задачи (бинарная классификация). Оно предлагает надежный стекинг моделей (GBM, XGBoost, GLM, DRF, DeepLearning), автоматическую кросс-валидацию, раннюю остановку и поддержку production-сценариев. Несмотря на отсутствие поддержки мультимодальности и GPU для всех моделей, оно хорошо подходит для задач классификации с высокой воспроизводимостью и стабильностью, что особенно важно в корпоративных и промышленных условиях. В отличие от AutoGluon, которое требует больше ресурсов и может быть менее предсказуемо на небольших наборах данных, H2O AutoML обеспечивает более предсказуемое поведение и более широкую совместимость с существующими инфраструктурными решениями.

### Generated Prompt
input_variables=['analysis_json', 'frameworks_list', 'metadata_json'] input_types={} partial_variables={} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='Ты — эксперт по AutoML и ML-инженер. Твоя задача — выбрать наиболее подходящий AutoML-фреймворк для табличных данных. Оцени качество данных, размер, тип задачи, ограничения и предложи лучший вариант из списка.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['analysis_json', 'frameworks_list', 'metadata_json'], input_types={}, partial_variables={}, template='Вот анализ датасета и метаданные:\n\n### 📊 Data Analysis\n{analysis_json}\n\n### 🧾 Metadata\n{metadata_json}\n\n### ⚙️ Доступные AutoML фреймворки\n{frameworks_list}\n\nПоясни свой выбор кратко, но содержательно. Если несколько подходят, выбери наиболее универсальный и стабильный вариант. Ответ верни в формате JSON:\n{{"framework": "...", "reason": "..."}}'), additional_kwargs={})]


In [7]:
print(type(prompt))

<class 'langchain_core.prompts.chat.ChatPromptTemplate'>


# Кодогенерация